In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("dataset.csv")

In [3]:
data.head()

,active,amenities,balconies,bathroom,combineDescription,completeStreetName,deposit,facing,facingDesc,floor,...,property_size,reactivationSource,rent_amount,sharedAccomodation,shortUrl,swimmingPool,totalFloor,type_bhk,waterSupply,weight
0,True,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",3,3,NaN,"Shreya carnation, Block I, NCB Enclave, Gachib...",90000,W,West,3,...,2200,USER_DASHBOARD,28000,False,http://nobr.kr/sv/9Kvqj,False,5,BHK3,CORP_BORE,NaN
1,True,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,2,NaN,"Inner Ring Rd, near RTO Bandlaguda South Zone",45000,E,East,2,...,1200,USER_DASHBOARD,15000,False,http://nobr.kr/sv/j2pD6,False,2,BHK3,BOREWELL,NaN
2,True,"{""LIFT"":true,""GYM"":true,""INTERNET"":false,""AC"":...",3,3,NaN,"Rd Number 2, Shirdi Sai Nagar, Manikonda, Hyde...",80000,E,East,0,...,1800,MISSED_CALL,16000,False,http://nobr.kr/sv/TXJmj,False,3,BHK3,CORP_BORE,NaN
3,True,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,2,NaN,"Plot No. 44, Road No. 1/A, kakatiya colony, LB...",18000,W,West,2,...,750,MISSED_CALL,9000,False,http://nobr.kr/sv/FnCsf,False,2,BHK2,CORP_BORE,NaN
4,True,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",2,2,NaN,"Madhapur HUDA Techno Enclave, Near MaxCure Su...",80000,E,East,2,...,1250,None,32500,False,http://nobr.kr/sv/BQFRb,False,5,BHK2,CORP_BORE,NaN


In [4]:
data.shape

(19110, 36)

## Preprocessing

##### Extracting information out of amenities column as it is one of the important factors in deciding price values followed by getting rid of unwanted columns

In [5]:
amenities_columns = ['LIFT', 'GYM', 'INTERNET', 'AC', 'CLUB', 'INTERCOM', 'POOL', 'CPA',
                     'FS', 'SERVANT', 'SECURITY', 'SC', 'GP', 'PARK', 'RWH', 'STP',
                     'HK', 'PB', 'VP']

In [6]:
for column in amenities_columns:
    data[column] = data['amenities'].str.extract(f"({column}:(True|False))", expand=True)[1].astype(bool).astype('int64')

In [7]:
df = data.drop(['amenities','balconies','active','combineDescription','completeStreetName','facing','id','gym','lift','localityId','location','ownerName','parkingDesc','propertyTitle','reactivationSource','shortUrl','type_bhk','weight'], axis=1)

In [8]:
df = df.replace('None', pd.NA).dropna()

##### Making sure that the data types of respective columns are suitable for EDA

In [9]:
df['maintenanceAmount'] = df['maintenanceAmount'].astype('int64')

In [10]:
df['swimmingPool'] = df['swimmingPool'].astype('int64')

#### Since total rent is a sum of the maintenance and usual rent amount, they have been collectively taken as one entity

In [11]:
df['totalprice'] = df[['maintenanceAmount', 'rent_amount']].apply(lambda x: x[x.notnull()].sum(), axis=1)

In [12]:
dff = df.drop(['maintenanceAmount','deposit','isMaintenance','loanAvailable','rent_amount','propertyType','floor','sharedAccomodation','FS','PARK','POOL','SC','CPA','GP','RWH','STP','HK','PB','VP'],axis=1)

In [13]:
dff.head()

,bathroom,facingDesc,furnishingDesc,locality,parking,property_age,property_size,swimmingPool,totalFloor,waterSupply,LIFT,GYM,INTERNET,AC,CLUB,INTERCOM,SERVANT,SECURITY,totalprice
0,3,West,Semi,Gachibowli,BOTH,5,2200,0,5,CORP_BORE,1,1,1,1,1,1,1,1,30000
2,3,East,Semi,Manikonda,BOTH,0,1800,0,3,CORP_BORE,1,1,1,1,1,1,1,1,17000
3,2,West,Unfurnished,LB Nagar,BOTH,0,750,0,2,CORP_BORE,1,1,1,1,1,1,1,1,9500
4,2,East,Semi,HITEC City,BOTH,5,1250,0,5,CORP_BORE,1,1,1,1,1,1,1,1,34500
5,3,North,Semi,Kondapur,BOTH,1,1517,1,6,CORP_BORE,1,1,1,1,1,1,1,1,35700


#### Now, it's looking better

In [14]:
dff.shape

(5240, 19)

### Some data cleaning

#### Cleaning the locality column of delimiters

In [15]:
data['locality'] = data['locality'].str.replace(',', '')

In [16]:
locality_counts = dff['locality'].value_counts()

In [17]:
df3 = dff[dff['locality'].isin(locality_counts[locality_counts > 10].index)]

#### Renaming the columns for more clarity

In [18]:
df4 = df3.rename(columns = {'furnishingDesc' : 'furnishing', 'LIFT' : 'lift', 'GYM' : 'gym','INTERNET' : 'internet', 'CLUB' : 'club', 'SERVANT' : 'servant', 'SECURITY' : 'security'})

In [19]:
df4.head()

,bathroom,facingDesc,furnishing,locality,parking,property_age,property_size,swimmingPool,totalFloor,waterSupply,lift,gym,internet,AC,club,INTERCOM,servant,security,totalprice
0,3,West,Semi,Gachibowli,BOTH,5,2200,0,5,CORP_BORE,1,1,1,1,1,1,1,1,30000
2,3,East,Semi,Manikonda,BOTH,0,1800,0,3,CORP_BORE,1,1,1,1,1,1,1,1,17000
3,2,West,Unfurnished,LB Nagar,BOTH,0,750,0,2,CORP_BORE,1,1,1,1,1,1,1,1,9500
5,3,North,Semi,Kondapur,BOTH,1,1517,1,6,CORP_BORE,1,1,1,1,1,1,1,1,35700
6,2,East,Semi,Gachibowli,BOTH,5,1198,1,24,CORP_BORE,1,1,1,1,1,1,1,1,33800


### One-Hot Encoding categorical columns

In [20]:
df5 = df4[(df4['bathroom'] < 5)]

In [21]:
categorical_columns = ['facingDesc', 'furnishing', 'locality', 'parking', 'waterSupply']

In [22]:
df_dummies = pd.get_dummies(df5[categorical_columns])

In [ ]:
df6 = pd.concat([df5.drop(columns=categorical_columns), df_dummies], axis=1)

In [23]:
df6 = pd.concat([df5.drop(columns=categorical_columns), df_dummies], axis=1)

In [24]:
cd = df6.drop(['swimmingPool', 'lift', 'gym', 'internet', 'AC', 'club', 'INTERCOM', 'servant', 'security'], axis=1)
cd

,bathroom,property_age,property_size,totalFloor,totalprice,facingDesc_Don't Know,facingDesc_East,facingDesc_North,facingDesc_North-East,facingDesc_North-West,...,locality_West Marredpally,locality_Whitefields,locality_Yousufguda,parking_BOTH,parking_FOUR_WHEELER,parking_NONE,parking_TWO_WHEELER,waterSupply_BOREWELL,waterSupply_CORPORATION,waterSupply_CORP_BORE
0,3,5,2200,5,30000,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,3,0,1800,3,17000,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,2,0,750,2,9500,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
5,3,1,1517,6,35700,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
6,2,5,1198,24,33800,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19001,2,5,1100,5,15000,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
19006,3,0,1735,8,34500,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
19007,2,5,1800,3,19000,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
19010,3,0,1782,27,46200,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


### Choosing data and target values

In [25]:
x = cd.drop('totalprice', axis = 1)
y = cd.totalprice

In [26]:
from sklearn.model_selection import train_test_split

#### Splitting the dataset for testing the model's accuracy

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=40)

## Exploring Various Regressors

#### Testing multiple Machine Learning models such as Linear Regression, L2 and L1 Regularization, SVM Regression, Decision Tree Regression, Random Forest Regression, Gradient Boosting Regrssion, XGBoost Regression and Neural Network model (Multi-Layer Perceptron)